# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [4]:
!python -V

Python 3.5.2


In [5]:
!pip list | grep tensorflow

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
tensorflow-gpu (1.0.0)


In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [7]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [8]:
import numpy as np
import problem_unittests as tests

from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    #convert to str from list and split (space word ' ' with a delimita)
    as_one = ' '.join(text) 
    
    #count where each word first occurs...to create an index....
    all_words = as_one.split()
    
    
    all_words = text
    counts = Counter(all_words)
    
    #get the unique words, and sort by most frequent
    complete_vocab = sorted(counts, key=counts.get, reverse=True)

    #for each word in the vocab give it a number, starting at 1 (so we leave ZERO free for NULL)
    #vocab_to_int = {single_word: ii for ii, single_word in enumerate(complete_vocab, 1)}    
    
    #int_to_vocab = {ii: single_word for ii, single_word in enumerate(complete_vocab, 1)} 
    
    
    # using offset above causes an error "train_loss = nan" So not offsetting

    vocab_to_int = {single_word: ii for ii, single_word in enumerate(complete_vocab)}    
    
    int_to_vocab = {ii: single_word for ii, single_word in enumerate(complete_vocab)} 

    return  vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [9]:
test_text = '''
Moe_Szyslak Moe's Tavern Where the elite meet to drink
Bart_Simpson Eh yeah hello is Mike there Last name Rotch
Moe_Szyslak Hold on I'll check Mike Rotch Mike Rotch Hey has anybody seen Mike Rotch lately
Moe_Szyslak Listen you little puke One of these days I'm gonna catch you and I'm gonna carve my name on your back with an ice pick
Moe_Szyslak Whats the matter Homer You're not your normal effervescent self
Homer_Simpson I got my problems Moe Give me another one
Moe_Szyslak Homer hey you should not drink to forget your problems
Barney_Gumble Yeah you should only drink to enhance your social skills'''


test_text = test_text.lower()
test_text = test_text.split()
vocab_to_int, int_to_vocab = create_lookup_tables(test_text)
        
print(vocab_to_int)
print('.........')
print(vocab_to_int)



{'i': 20, 'bart_simpson': 36, 'an': 23, 'little': 24, 'moe_szyslak': 0, 'homer': 7, 'should': 8, 'hey': 12, "you're": 25, 'forget': 26, 'rotch': 1, 'you': 2, 'self': 27, 'mike': 3, 'pick': 28, 'got': 29, 'one': 9, 'lately': 31, 'of': 32, 'days': 33, 'my': 10, 'meet': 34, 'name': 11, 'problems': 19, 'not': 13, 'puke': 43, 'with': 35, 'me': 22, 'back': 37, 'carve': 38, 'these': 39, 'to': 5, 'hello': 40, 'the': 14, 'has': 42, 'drink': 6, 'eh': 44, 'gonna': 15, 'on': 16, 'where': 21, 'hold': 46, 'is': 50, "moe's": 48, 'elite': 49, 'whats': 47, 'give': 51, "i'm": 17, 'last': 52, 'skills': 53, 'there': 54, 'anybody': 45, 'another': 55, 'matter': 56, 'and': 57, 'barney_gumble': 41, 'normal': 58, 'yeah': 18, 'check': 60, 'homer_simpson': 70, 'catch': 61, 'effervescent': 59, 'tavern': 62, 'your': 4, "i'll": 63, 'social': 64, 'ice': 65, 'seen': 66, 'moe': 67, 'only': 30, 'listen': 69, 'enhance': 68}
.........
{'i': 20, 'bart_simpson': 36, 'an': 23, 'little': 24, 'moe_szyslak': 0, 'homer': 7, 'sh

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [10]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
        
    # Tokenize dictionary
    # key   = punctuation e.g. "!"  
    # value = token       e.g. "||Exclamation_Mark||"
    symbols_dict = {'.':'||Period||', 
                    ',':"||Comma||", 
                    '"':"||Quotation_Mark||", 
                    ';':"||Semicolon||", 
                    '!':"||Exclamation_Mark||", 
                    '?':"||Question_Mark||", 
                    '(':"||Left_Parentheses||", 
                    ')':"||Right_Parentheses||", 
                    '--':"||Dash||", 
                    '\n':"||Return||"}
    
    return symbols_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [14]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    
    # TF wants 2D not 1D so we have [None, None] not just [None]
    inputs = tf.placeholder(tf.int32, [None, None], name="input")  
    
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
        
    learning_rate = tf.placeholder(tf.float32, name='learning_rate') #grrrrrrrrr <-- make sure always float...!
    
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [72]:
#Params for tuning

# Since keep_prob is a scalar (a 0-dimensional tensor), you shouldn't provide a size to tf.placeholder
keep_prob = 0.6
lstm_layers = 1

In [73]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    
    #basic LSTM cell (ref: Senitment notebook.ipynb)
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    # simple way to create multiple layers of LSTM cells 
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
        
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    # Apply the name "initial_state" to the initial state using tf.identity()
    state = tf.identity(initial_state, name = "initial_state")
    
    # return the stack of cells and the initial state
    return cell, state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [74]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
    # len(int_to_vocab) 6779, shows a smallish vocabulary, but inefficient to one-hot encode classes (see word2vec)
    # so use embedding layer as a lookup (no need to multiply)
    # https://www.youtube.com/watch?v=nirSPnZfEIw&feature=youtu.be&t=2m38s
    
    # Size of the embedding vectors (number of units in the embedding layer)
    #tf.random_uniform(shape, minval=0, maxval=None, 
    embedding = tf.Variable(tf.truncated_normal((vocab_size, embed_dim), stddev=0.20)) #gives a nice gradual drop
    
    ###### ALTERNATIVE
    # if we want to truncate the initial embeddings outside the stdv range:
    # The generated values follow a normal distribution with specified mean and standard deviation, 
    # except that values whose magnitude is more than 2 standard deviations from the mean are dropped and re-picked
    #embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1.0, 1.0))

    # This function takes the embedding matrix and an input tensor, such as the review vectors. 
    # Then, it'll return another tensor with the embedded vectors. 
    # So, if the embedding layer as 200 units, the function will return a tensor with size [batch_size, 200].
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [75]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    #Use tf.nn.dynamic_rnn to add the forward pass through the RNN. 
    # Remember that we're actually passing in vectors from the embedding layer
    
    # initial_state is the cell state that is passed between the hidden layers in successive time steps. 
    # tf.nn.dynamic_rnn takes care of most of the work for us. 
    # We pass in our cell and the input to the cell, then it does the unrolling and everything else for us. 
    # It returns outputs for each time step and the final_state of the hidden layer.


    # We want to predict the next word, so we want to use all the outputs
    # Creates a recurrent neural network specified by RNNCell cell.
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype = tf.float32)

    final = tf.identity(final_state, name = "final_state")

    return outputs, final


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [76]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    # apply embedding (dimensions = rnn_size)
    embeddings = get_embed(input_data, vocab_size, rnn_size)
    
    # build the rnn
    output, final_state = build_rnn(cell, embeddings)    
    
    #logits or weights :)
    # representing a fully connected weight matrix, 
    # which is multiplied by the inputs to produce a Tensor of hidden units
    # Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs

    # set activation to tanh? default is ReLU just linear
    # set normalizer to batch_norm? Otherwise just biases applied
    # set weights_initializer to: Xavier Glorot and Yoshua Bengio (2010): Understanding the difficulty 
    #              of training deep feedforward  neural networks. International conference on artificial 
    #              intelligence and statistics.
    
    # tf.nn.tanh - https://www.tensorflow.org/api_docs/python/tf/tanh
    # predictions = tf.contrib.layers.fully_connected(cell, 1, activation_fn=tf.sigmoid)
    # output need to match our total vocab, as could be any of them :)

                                                    #weights_initializer = tf.truncated_normal_initializer(stddev=0.1), 
                                                    #biases_initializer=tf.zeros_initializer())
    
    predictions = tf.contrib.layers.fully_connected(output, 
                                                    vocab_size,
                                                    activation_fn=None, #tf.nn.tanh <-- slows the conversion by 40%
                                                    weights_initializer=tf.contrib.layers.xavier_initializer_conv2d(uniform=True, 
                                                                                                                    seed=None, 
                                                                                                                    dtype=tf.float32))
    
    
    return predictions, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [77]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    # Mat's video - https://nd101.slack.com/files/mat/F4P1VE8SY/sequence_batches.mp4
    # based on batching in Sentiment_RNN_Solution.ipynb
    #(1)So sequence is the text....
    #(2)Batch size is how many sequences we want to train on simultaniously
    #(3)sq_length is number of inputs, so we can keep data together...
    
    # obviously taget is just what we want to achieve...from give input, so easy to achieve with an off set of +1
    
    
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    
    #The first element is a single batch of input with the shape [batch size, sequence length]
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])

    
    #The second element is a single batch of targets with the shape [batch size, sequence length]
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    #print(x_batches)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    #print(y_batches)
    
    return np.array(list(zip(x_batches, y_batches)))
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [91]:
# Number of Epochs
num_epochs = 88 #or we start to over fit
# Batch Size
batch_size = 256 # swap to lower
# RNN Size
rnn_size = 512
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = int(len(int_text) / (batch_size * seq_length)) 


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [92]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [93]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/13   train_loss = 8.820
Epoch   1 Batch    0/13   train_loss = 5.680
Epoch   2 Batch    0/13   train_loss = 5.015
Epoch   3 Batch    0/13   train_loss = 4.641
Epoch   4 Batch    0/13   train_loss = 4.321
Epoch   5 Batch    0/13   train_loss = 4.067
Epoch   6 Batch    0/13   train_loss = 3.826
Epoch   7 Batch    0/13   train_loss = 3.619
Epoch   8 Batch    0/13   train_loss = 3.377
Epoch   9 Batch    0/13   train_loss = 3.170
Epoch  10 Batch    0/13   train_loss = 2.988
Epoch  11 Batch    0/13   train_loss = 2.842
Epoch  12 Batch    0/13   train_loss = 2.680
Epoch  13 Batch    0/13   train_loss = 2.561
Epoch  14 Batch    0/13   train_loss = 2.483
Epoch  15 Batch    0/13   train_loss = 2.388
Epoch  16 Batch    0/13   train_loss = 2.224
Epoch  17 Batch    0/13   train_loss = 2.126
Epoch  18 Batch    0/13   train_loss = 2.014
Epoch  19 Batch    0/13   train_loss = 1.908
Epoch  20 Batch    0/13   train_loss = 1.813
Epoch  21 Batch    0/13   train_loss = 1.732
Epoch  22 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [94]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [95]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [96]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [97]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    # ref: intro-to-rnns/Anna_KaRNNa.ipynb - we are choosing words not letters
    
    n_most_probable_words = 5 #[5..10]  to remove nonsensical words
    
    #reduce the dimensions
    p = np.squeeze(probabilities)

    #sort
    p[np.argsort(p)[:-n_most_probable_words]] = 0.0
    
    # make it out of 1
    p = p / np.sum(p)

    #get a random word from our set of high probable words
    c = np.random.choice(len(probabilities), 1, p=p)[0]
    
    #select the word from the dictionary
    return int_to_vocab[c]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)


Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [98]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: or maybe it's groveling that works. eh, six of one.


barflies:(hurt sigh) just ask your daughter to be a little about it?
moe_szyslak: aw, that broad is powered by non-american workers die.
moe_szyslak: no. not yet at" look at ease?"
moe_szyslak: boy, ya remember that, so you can spit!


lenny_leonard: hey, join the party!" weekly reader star selection" good things to make with a guess....
football_announcer: touchdown! the insurance company pays off five thousand clams. i keep showing renee with the jay leno show?
betty: uh, that's okay.
homer_simpson: oh, right-- like this the neck i'm other day i got the bad guy.
homer_simpson: and you're the greatest news involving multiple normal.
ned_flanders: i got somethin' of course, i. your tolerance is too high.
dejected_barfly:(a little coffee'll do a day a guy who fixes to fix all that stuff.
duffman: duffman is it who does


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.